## Assignment 2: Set up the work for data-driven inference based on CitiBike data
###### Author: Daniel Fay
   * Write a Jupyter Notebook on compute. This will require you to use the JupyterHub (instructions here ). Write a notebook thats states the question you want to ask, and formulates the Null and Alternative hypothesis (remember the confidence level!)
   * Use pandas to read in the CitiBike files, either from the DF, or locally, but you must be able to download them on the spot (so the TA can reproduce your work).
   * Display the top few rows of the DF in your notebook. This table must be rendered.
   * Display the reducted dataframe. This table must be rendered.
   * Plot your data distributions.

In [5]:
# Import necessary packages and get environment variables
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

os.getenv('PUIDATA')

'/home/cusp/df1383/PUIdata'

In [6]:
# Wrangle Citibike Data (code used from Federica's sample notebook)
def get_citibike_data(date):
    os.system("curl -O https://s3.amazonaws.com/tripdata/" + date + "-citibike-tripdata.zip")
    os.system("unzip " + date + "-citibike-tripdata.zip")

date = '201606'
get_citibike_data(date)

In [7]:
# Read in CSV file into pandas dataframe and display top 10 rows
df = pd.read_csv(date+'-citibike-tripdata.csv')
df.head(10)

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,1470,6/1/2016 00:00:18,6/1/2016 00:24:48,380,W 4 St & 7 Ave S,40.734011,-74.002939,3236,W 42 St & Dyer Ave,40.758985,-73.993800,19859,Subscriber,1972.0,1
1,229,6/1/2016 00:00:20,6/1/2016 00:04:09,3092,Berry St & N 8 St,40.719009,-73.958525,3103,N 11 St & Wythe Ave,40.721533,-73.957824,16233,Subscriber,1967.0,1
2,344,6/1/2016 00:00:21,6/1/2016 00:06:06,449,W 52 St & 9 Ave,40.764618,-73.987895,469,Broadway & W 53 St,40.763441,-73.982681,22397,Subscriber,1989.0,1
3,1120,6/1/2016 00:00:28,6/1/2016 00:19:09,522,E 51 St & Lexington Ave,40.757148,-73.972078,401,Allen St & Rivington St,40.720196,-73.989978,16231,Subscriber,1991.0,1
4,229,6/1/2016 00:00:53,6/1/2016 00:04:42,335,Washington Pl & Broadway,40.729039,-73.994046,285,Broadway & E 14 St,40.734546,-73.990741,15400,Subscriber,1989.0,1
5,946,6/1/2016 00:01:01,6/1/2016 00:16:48,503,E 20 St & Park Ave,40.738274,-73.987520,495,W 47 St & 10 Ave,40.762699,-73.993012,25193,Subscriber,1974.0,1
6,2351,6/1/2016 00:01:05,6/1/2016 00:40:16,533,Broadway & W 39 St,40.752996,-73.987216,386,Centre St & Worth St,40.714948,-74.002345,19538,Subscriber,1986.0,1
7,773,6/1/2016 00:01:24,6/1/2016 00:14:18,492,W 33 St & 7 Ave,40.750200,-73.990931,483,E 12 St & 3 Ave,40.732233,-73.988900,17101,Subscriber,1986.0,1
8,1929,6/1/2016 00:01:27,6/1/2016 00:33:37,525,W 34 St & 11 Ave,40.755942,-74.002116,306,Cliff St & Fulton St,40.708235,-74.005301,17802,Subscriber,1968.0,1
9,725,6/1/2016 00:01:38,6/1/2016 00:13:43,2002,Wythe Ave & Metropolitan Ave,40.716887,-73.963198,3083,Bushwick Ave & Powers St,40.712477,-73.941000,21421,Subscriber,1971.0,2
